In [ ]:
# 미션 : 수정없이 이해 후 주석달기 / 혼자 or 둘
# 주석 : 양이 적어도 괜찮음

### 전국 민간 아파트 분양 가격 동향 자료 전처리 및 분석

- 데이터 : 공공데이터 포탈에서 제공, 2015년 10월 부터 2018년 7월까지의 주택분양 가격 정보 제공
- 참고 - 파일명 수정된 상태(한글 파일 -> 영문과 숫자)
- 한국 데이터셋 파일들 : 인코딩 수정 또는 확인(cp949, euc-kr : utf-8형식으로 변환 및 재저장)
- 단순 경고등이 발생(skip 명령어 적용)

csv file

1. avg15.09.csv - 2013년 12월 ~ 2015년 9월까지의 자료 <br>
2. avg18.07.csv - 2015년 10월 ~ 2018년 7월까지의 자료

### 피벗 Table
- 피벗이란? 회전하는 물체의 균형을 잡아주는 중심점(원의 중심정과 흡사)
- 피벗 테이블이란? 기존 데이터의 칼럼을 재구성해서 데이터에 대한 통계를 한눈에 파악 할 수 있게 정리한 표
- 피벗 기능 활용시 장점 : 데이터를 원하는 형태로 손쉽게 집계 할 수 있음
- API 소개
```
pd.pivot_table(df,  # 피벗할 데이터프레임
              index = '행 위치에 들어갈 열',
              columns = '열 위치에 들어갈 열',
              values = '데이터로 사용할 열',
              aggfunc = 데이터 집계함수)


pd.pivot_table(df,  # 피벗할 데이터프레임
              index = '행 위치에 들어갈 열',
              columns = '열 위치에 들어갈 열',
              values = '데이터로 사용할 열',
              aggfunc = 데이터 집계함수)


data = {"소속": ["jyp", "psy", "jyp", "jyp", "jyp", "jyp", "psy", "psy", "psy"],
        "가수": ["비", "제시", "박진형", "원더걸스", "제니", "샤샤", "현아", "싸이", "하니"],
        "성별": ['남자', '여자', '남자', '여자', '여자', '여자', '여자', '남자', '여자'],
        "연령": [15, 20, 20, 30, 30, 40, 50, 60, 70]}
data
```


In [2]:
#단순 경고 메세지 skip할수 있게 하는 설정
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import re

2015년 1월~18년 7월까지의 신규 아파트 분양 정보 확인 및 전처리
(평당 분양가격등도 추가)

In [22]:
#avg18.07.csv 파일을 read : 변수명 pre_sale

pre_sale = pd.read_csv('dataset/aptPriceDataSet/avg18.07.csv', encoding="utf-8")

pre_sale

,지역명,규모구분,연도,월,분양가격(㎡)
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721
4,서울,전용면적 102㎡초과,2015,10,5879
...,...,...,...,...,...
2885,제주,전체,2018,7,3334
2886,제주,전용면적 60㎡이하,2018,7,NaN
2887,제주,전용면적 60㎡초과 85㎡이하,2018,7,3356
2888,제주,전용면적 85㎡초과 102㎡이하,2018,7,3226


In [23]:
'''
1. 5개 head
2. 5개 tail
3. null 보유 여부 확인 
4. shape 로 row수? columns수?
'''
pre_sale.head(10)

,지역명,규모구분,연도,월,분양가격(㎡)
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721
4,서울,전용면적 102㎡초과,2015,10,5879
5,인천,전체,2015,10,3163
6,인천,전용면적 60㎡이하,2015,10,3488
7,인천,전용면적 60㎡초과 85㎡이하,2015,10,3119
8,인천,전용면적 85㎡초과 102㎡이하,2015,10,3545
9,인천,전용면적 102㎡초과,2015,10,3408


In [24]:
# 분양 가격에 결측치 존재

pre_sale.tail()

,지역명,규모구분,연도,월,분양가격(㎡)
2885,제주,전체,2018,7,3334
2886,제주,전용면적 60㎡이하,2018,7,NaN
2887,제주,전용면적 60㎡초과 85㎡이하,2018,7,3356
2888,제주,전용면적 85㎡초과 102㎡이하,2018,7,3226
2889,제주,전용면적 102㎡초과,2018,7,2990


In [25]:
print(pre_sale.shape)


# 분양 가격에는 null값 보유 따라서 null 개수를 제외한 유효한 데이터 개수 확인
print(pre_sale.info())

(2890, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2890 entries, 0 to 2889
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   지역명      2890 non-null   object
 1   규모구분     2890 non-null   object
 2   연도       2890 non-null   int64 
 3   월        2890 non-null   int64 
 4   분양가격(㎡)  2750 non-null   object
dtypes: int64(2), object(3)
memory usage: 113.0+ KB
None


In [26]:
# 해당 컬럼의 null 개수의 합

pre_sale['지역명'].isnull().sum()

0

In [27]:
pre_sale['분양가격(㎡)'].isnull().sum()

140

In [28]:
#각 컬럼명 type 확인 
pre_sale.dtypes

지역명        object
규모구분       object
연도          int64
월           int64
분양가격(㎡)    object
dtype: object

In [29]:
pre_sale.isnull()

,지역명,규모구분,연도,월,분양가격(㎡)
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
2885,False,False,False,False,False
2886,False,False,False,False,True
2887,False,False,False,False,False
2888,False,False,False,False,False


In [30]:
# 분양가격(㎡) 값만 검색해 보기

pre_sale['분양가격(㎡)']

0       5841
1       5652
2       5882
3       5721
4       5879
        ... 
2885    3334
2886     NaN
2887    3356
2888    3226
2889    2990
Name: 분양가격(㎡), Length: 2890, dtype: object

In [31]:
# 분양가격(㎡)으로 새로운 컬럼을 도출(연산도 수행 가능한 타입으로 변환)
# raw 데이터는 유지, 새로운 컬럼을 추가(분양가격, 평당분양가격)

# 존재하는 컬럼으로 새로운 Series 객체 생성
pre_sale_price = pre_sale['분양가격(㎡)']
pre_sale_price

0       5841
1       5652
2       5882
3       5721
4       5879
        ... 
2885    3334
2886     NaN
2887    3356
2888    3226
2889    2990
Name: 분양가격(㎡), Length: 2890, dtype: object

In [32]:
# 분양 가격 적용 전

pre_sale.head()

,지역명,규모구분,연도,월,분양가격(㎡)
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721
4,서울,전용면적 102㎡초과,2015,10,5879


분양 가격 적용을 숫자 포멧으로 적용

- 분양가격(㎡) 타입이 숫자가 아니기 때문에 연산 불가
- 분양가격 컬럼 추가하면서 숫자화

In [33]:
pre_sale['분양가격(㎡)']

0       5841
1       5652
2       5882
3       5721
4       5879
        ... 
2885    3334
2886     NaN
2887    3356
2888    3226
2889    2990
Name: 분양가격(㎡), Length: 2890, dtype: object

In [34]:
pre_sale['분양가격(㎡)'] * 3.3

TypeError: can't multiply sequence by non-int of type 'float'

In [35]:
#분양가격 새로운 컬럼 생성(타입은 숫자 왜? 연산 수행)
#숫자 타입인 분양 가격을 기반으로 평당분양가격 컬럼 생성
# * 3.3
'''
   errors="coerce" : 에러가 발생시 무시
   errors="raise: 에러 발생시 표현   
'''
pre_sale['분양가격'] = pd.to_numeric(pre_sale_price, errors='coerce')

In [36]:
pre_sale.head()

,지역명,규모구분,연도,월,분양가격(㎡),분양가격
0,서울,전체,2015,10,5841,5841.0
1,서울,전용면적 60㎡이하,2015,10,5652,5652.0
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882,5882.0
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721,5721.0
4,서울,전용면적 102㎡초과,2015,10,5879,5879.0


In [37]:
pre_sale['분양가격'].dtype

dtype('float64')

In [38]:
pre_sale['분양가격(㎡)'].dtype

dtype('O')

In [39]:
type(pre_sale['분양가격(㎡)'])

pandas.core.series.Series

In [40]:
pre_sale

,지역명,규모구분,연도,월,분양가격(㎡),분양가격
0,서울,전체,2015,10,5841,5841.0
1,서울,전용면적 60㎡이하,2015,10,5652,5652.0
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882,5882.0
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721,5721.0
4,서울,전용면적 102㎡초과,2015,10,5879,5879.0
...,...,...,...,...,...,...
2885,제주,전체,2018,7,3334,3334.0
2886,제주,전용면적 60㎡이하,2018,7,NaN,NaN
2887,제주,전용면적 60㎡초과 85㎡이하,2018,7,3356,3356.0
2888,제주,전용면적 85㎡초과 102㎡이하,2018,7,3226,3226.0


- 평당분양가격 이라는 새로운 컬럼 생성해서 추가 
- 분양가격 * 3.3 연산 

In [41]:
pre_sale['평당분양가격'] = pre_sale['분양가격']*3.3

In [42]:
pre_sale['평당분양가격'].head()

0    19275.3
1    18651.6
2    19410.6
3    18879.3
4    19400.7
Name: 평당분양가격, dtype: float64

In [43]:
pre_sale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2890 entries, 0 to 2889
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지역명      2890 non-null   object 
 1   규모구분     2890 non-null   object 
 2   연도       2890 non-null   int64  
 3   월        2890 non-null   int64  
 4   분양가격(㎡)  2750 non-null   object 
 5   분양가격     2658 non-null   float64
 6   평당분양가격   2658 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 158.2+ KB


In [44]:
# 결측치를 단순 메모장에서 필수로 확인
# 분양가격, 평당분양가격 컬럼은 공백도 null로 집계

pre_sale['분양가격(㎡)'].isnull().sum()

140

In [45]:
pre_sale['분양가격'].isnull().sum()

232

In [46]:
pre_sale['평당분양가격'].isnull().sum()

232

In [47]:
pre_sale.dtypes

지역명         object
규모구분        object
연도           int64
월            int64
분양가격(㎡)     object
분양가격       float64
평당분양가격     float64
dtype: object

- describe()<br><br>

메소드는 생성했던 DataFrame의 간단한 통계 정보<br>
컬럼별로 데이터의 개수(count), 데이터의 평균 값(mean), 표준 편차(std), 최솟값(min), 4분위수(25%, 50%, 75%), 최댓값(max)들의 정보를 알 수 있음

In [48]:
pre_sale.describe()

,연도,월,분양가격,평당분양가격
count,2890.000000,2890.000000,2658.000000,2658.000000
mean,2016.676471,6.382353,3063.297216,10108.880813
std,0.898539,3.515406,1112.743769,3672.054436
min,2015.000000,1.000000,1868.000000,6164.400000
25%,2016.000000,3.000000,2348.000000,7748.400000
50%,2017.000000,6.000000,2743.000000,9051.900000
75%,2017.000000,10.000000,3293.750000,10869.375000
max,2018.000000,12.000000,8098.000000,26723.400000


In [49]:
pre_sale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2890 entries, 0 to 2889
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지역명      2890 non-null   object 
 1   규모구분     2890 non-null   object 
 2   연도       2890 non-null   int64  
 3   월        2890 non-null   int64  
 4   분양가격(㎡)  2750 non-null   object 
 5   분양가격     2658 non-null   float64
 6   평당분양가격   2658 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 158.2+ KB


In [50]:
# object 타입에 한해서만 기초 통계
pre_sale.describe(include=[np.int64])

,연도,월
count,2890.000000,2890.000000
mean,2016.676471,6.382353
std,0.898539,3.515406
min,2015.000000,1.000000
25%,2016.000000,3.000000
50%,2017.000000,6.000000
75%,2017.000000,10.000000
max,2018.000000,12.000000


In [51]:
pre_sale.describe(include=[np.float64])

,분양가격,평당분양가격
count,2658.000000,2658.000000
mean,3063.297216,10108.880813
std,1112.743769,3672.054436
min,1868.000000,6164.400000
25%,2348.000000,7748.400000
50%,2743.000000,9051.900000
75%,3293.750000,10869.375000
max,8098.000000,26723.400000


In [52]:
pre_sale.describe(include=[np.object])

,지역명,규모구분,분양가격(㎡)
count,2890,2890,2750
unique,17,5,1305
top,부산,전용면적 85㎡초과 102㎡이하,2221
freq,170,578,17


In [53]:
pre_sale

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
0,서울,전체,2015,10,5841,5841.0,19275.3
1,서울,전용면적 60㎡이하,2015,10,5652,5652.0,18651.6
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882,5882.0,19410.6
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721,5721.0,18879.3
4,서울,전용면적 102㎡초과,2015,10,5879,5879.0,19400.7
...,...,...,...,...,...,...,...
2885,제주,전체,2018,7,3334,3334.0,11002.2
2886,제주,전용면적 60㎡이하,2018,7,NaN,NaN,NaN
2887,제주,전용면적 60㎡초과 85㎡이하,2018,7,3356,3356.0,11074.8
2888,제주,전용면적 85㎡초과 102㎡이하,2018,7,3226,3226.0,10645.8


- 연도 기준으로 2017년 데이터 검색

In [54]:
pre_sale['연도']

0       2015
1       2015
2       2015
3       2015
4       2015
        ... 
2885    2018
2886    2018
2887    2018
2888    2018
2889    2018
Name: 연도, Length: 2890, dtype: int64

In [55]:
# 연도가 2017년도 데이터 존재 여부 확인

pre_sale['연도']==2017

0       False
1       False
2       False
3       False
4       False
        ...  
2885    False
2886    False
2887    False
2888    False
2889    False
Name: 연도, Length: 2890, dtype: bool

In [56]:
# 2017년도 관련 데이터만 검색

pre_sale_2017 = pre_sale.loc[pre_sale['연도'] == 2017]

pre_sale_2017

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
1275,서울,전체,2017,1,6450,6450.0,21285.0
1276,서울,전용면적 60㎡이하,2017,1,6662,6662.0,21984.6
1277,서울,전용면적 60㎡초과 85㎡이하,2017,1,6500,6500.0,21450.0
1278,서울,전용면적 85㎡초과 102㎡이하,2017,1,7030,7030.0,23199.0
1279,서울,전용면적 102㎡초과,2017,1,6771,6771.0,22344.3
...,...,...,...,...,...,...,...
2290,제주,전체,2017,12,3578,3578.0,11807.4
2291,제주,전용면적 60㎡이하,2017,12,5380,5380.0,17754.0
2292,제주,전용면적 60㎡초과 85㎡이하,2017,12,3467,3467.0,11441.1
2293,제주,전용면적 85㎡초과 102㎡이하,2017,12,NaN,NaN,NaN


In [57]:
# 2017년도 데이터 row, column 수 검색

pre_sale_2017.shape

(1020, 7)

In [58]:
pre_sale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2890 entries, 0 to 2889
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지역명      2890 non-null   object 
 1   규모구분     2890 non-null   object 
 2   연도       2890 non-null   int64  
 3   월        2890 non-null   int64  
 4   분양가격(㎡)  2750 non-null   object 
 5   분양가격     2658 non-null   float64
 6   평당분양가격   2658 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 158.2+ KB


In [59]:
# 지역명별 개수
# select distinct 지역명 from pre_sale

pre_sale['지역명'].value_counts()

부산    170
울산    170
경기    170
광주    170
전남    170
충북    170
인천    170
경남    170
대구    170
세종    170
서울    170
강원    170
경북    170
전북    170
충남    170
제주    170
대전    170
Name: 지역명, dtype: int64

In [60]:
pre_sale

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
0,서울,전체,2015,10,5841,5841.0,19275.3
1,서울,전용면적 60㎡이하,2015,10,5652,5652.0,18651.6
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882,5882.0,19410.6
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721,5721.0,18879.3
4,서울,전용면적 102㎡초과,2015,10,5879,5879.0,19400.7
...,...,...,...,...,...,...,...
2885,제주,전체,2018,7,3334,3334.0,11002.2
2886,제주,전용면적 60㎡이하,2018,7,NaN,NaN,NaN
2887,제주,전용면적 60㎡초과 85㎡이하,2018,7,3356,3356.0,11074.8
2888,제주,전용면적 85㎡초과 102㎡이하,2018,7,3226,3226.0,10645.8


### 전국 평균 분양 가격

In [61]:
pre_sale.groupby(pre_sale.연도)

In [62]:
pre_sale.groupby(pre_sale.연도).describe()

월                                                   분양가격  \
       count  mean       std   min    25%   50%    75%   max  count   
연도                                                                    
2015   255.0  11.0  0.818102  10.0  10.00  11.0  12.00  12.0  243.0   
2016  1020.0   6.5  3.453746   1.0   3.75   6.5   9.25  12.0  984.0   
2017  1020.0   6.5  3.453746   1.0   3.75   6.5   9.25  12.0  899.0   
2018   595.0   4.0  2.001683   1.0   2.00   4.0   6.00   7.0  532.0   

                   ...                  평당분양가격                             \
             mean  ...      75%     max  count          mean          std   
연도                 ...                                                      
2015  2788.707819  ...  3025.50  7092.0  243.0   9202.735802  3223.641902   
2016  2934.250000  ...  3148.50  8096.0  984.0   9683.025000  3535.757150   
2017  3139.541713  ...  3456.00  7887.0  899.0  10360.487653  3655.030071   
2018  3298.567669  ...  3647.25  8098.0  532.0  10885.273308  3946.651548   

                                                  
         min     25%     50%        75%      max  
연도                                                
2015  6164.4  7342.5  8302.8   9984.150  23403.6  
2016  6270.0  7530.6  8817.6  10390.050  26716.8  
2017  6520.8  7804.5  9401.7  11404.800  26027.1  
2018  6850.8  8154.3  9616.2  12035.925  26723.4  

[4 rows x 24 columns]

In [63]:
#연도별 수치 타입인 분양가격과 평균분양가격 집계정보 확인
# .T 속성은 DataFrame 에서 index 와 column 을 바꾼 형태의 DataFrame

pre_sale.groupby(pre_sale.연도).describe().T

연도                    2015          2016          2017          2018
월      count    255.000000   1020.000000   1020.000000    595.000000
       mean      11.000000      6.500000      6.500000      4.000000
       std        0.818102      3.453746      3.453746      2.001683
       min       10.000000      1.000000      1.000000      1.000000
       25%       10.000000      3.750000      3.750000      2.000000
       50%       11.000000      6.500000      6.500000      4.000000
       75%       12.000000      9.250000      9.250000      6.000000
       max       12.000000     12.000000     12.000000      7.000000
분양가격   count    243.000000    984.000000    899.000000    532.000000
       mean    2788.707819   2934.250000   3139.541713   3298.567669
       std      976.861182   1071.441561   1107.584870   1195.955014
       min     1868.000000   1900.000000   1976.000000   2076.000000
       25%     2225.000000   2282.000000   2365.000000   2471.000000
       50%     2516.000000   2672.000000   2849.000000   2914.000000
       75%     3025.500000   3148.500000   3456.000000   3647.250000
       max     7092.000000   8096.000000   7887.000000   8098.000000
평당분양가격 count    243.000000    984.000000    899.000000    532.000000
       mean    9202.735802   9683.025000  10360.487653  10885.273308
       std     3223.641902   3535.757150   3655.030071   3946.651548
       min     6164.400000   6270.000000   6520.800000   6850.800000
       25%     7342.500000   7530.600000   7804.500000   8154.300000
       50%     8302.800000   8817.600000   9401.700000   9616.200000
       75%     9984.150000  10390.050000  11404.800000  12035.925000
       max    23403.600000  26716.800000  26027.100000  26723.400000

In [64]:
pre_sale

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
0,서울,전체,2015,10,5841,5841.0,19275.3
1,서울,전용면적 60㎡이하,2015,10,5652,5652.0,18651.6
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882,5882.0,19410.6
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721,5721.0,18879.3
4,서울,전용면적 102㎡초과,2015,10,5879,5879.0,19400.7
...,...,...,...,...,...,...,...
2885,제주,전체,2018,7,3334,3334.0,11002.2
2886,제주,전용면적 60㎡이하,2018,7,NaN,NaN,NaN
2887,제주,전용면적 60㎡초과 85㎡이하,2018,7,3356,3356.0,11074.8
2888,제주,전용면적 85㎡초과 102㎡이하,2018,7,3226,3226.0,10645.8


### 피벗(Pivot) 테이블이란 많은 양의 데이터에서 필요한 자료만을 뽑아 새롭게 표를 작성해 주는 기능입니다

피벗 table 이해를 위한 docString 예제

In [65]:
#pre_sale.pivot_table(values=None, index=None, columns=None)
pre_sale

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
0,서울,전체,2015,10,5841,5841.0,19275.3
1,서울,전용면적 60㎡이하,2015,10,5652,5652.0,18651.6
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882,5882.0,19410.6
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721,5721.0,18879.3
4,서울,전용면적 102㎡초과,2015,10,5879,5879.0,19400.7
...,...,...,...,...,...,...,...
2885,제주,전체,2018,7,3334,3334.0,11002.2
2886,제주,전용면적 60㎡이하,2018,7,NaN,NaN,NaN
2887,제주,전용면적 60㎡초과 85㎡이하,2018,7,3356,3356.0,11074.8
2888,제주,전용면적 85㎡초과 102㎡이하,2018,7,3226,3226.0,10645.8


In [67]:
table = pre_sale.pivot_table(values='분양가격', index=['지역명'], columns=['규모구분'])
table.reset_index()

규모구분,지역명,전용면적 102㎡초과,전용면적 60㎡이하,전용면적 60㎡초과 85㎡이하,전용면적 85㎡초과 102㎡이하,전체
0,강원,2393.545455,2180.424242,2190.393939,2328.960000,2180.848485
1,경기,4135.333333,3752.939394,3542.757576,3675.878788,3547.181818
2,경남,2795.787879,2528.181818,2457.636364,2976.333333,2465.666667
3,경북,2610.878788,2321.666667,2330.454545,2659.480000,2332.212121
4,광주,3120.148148,2581.142857,2780.454545,2817.000000,2777.818182
5,대구,3737.666667,3419.606061,3312.333333,2905.800000,3302.030303
6,대전,4328.000000,2666.121212,2668.878788,2738.615385,2690.878788
7,부산,3815.090909,3244.484848,3391.272727,3345.060606,3422.696970
8,서울,6684.303030,6680.515152,6297.242424,7198.303030,6414.818182
9,세종,2815.666667,2759.620690,2809.636364,2801.484848,2807.060606


In [47]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})

In [48]:
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [49]:
table = df.pivot_table(values='D', index=['A', 'B'],
                       columns=['C'], aggfunc=np.sum)
table

C        large  small
A   B                
bar one    4.0    5.0
    two    7.0    6.0
foo one    4.0    1.0
    two    NaN    6.0

In [50]:
table = df.pivot_table(values=['D', 'E'], index=['A', 'C'],
                       aggfunc={'D': np.mean, 'E': np.mean})
table

D         E
A   C                        
bar large  5.500000  7.500000
    small  5.500000  8.500000
foo large  2.000000  4.500000
    small  2.333333  4.333333

In [51]:
table = df.pivot_table(values=['D', 'E'], index=['A', 'C'],  
                       aggfunc={'D': np.mean, 'E': np.mean})
table

D         E
A   C                        
bar large  5.500000  7.500000
    small  5.500000  8.500000
foo large  2.000000  4.500000
    small  2.333333  4.333333

In [52]:
table = df.pivot_table(values=['D', 'E'], index=['A', 'C'],  
                       aggfunc={'D': np.mean, 'E': np.mean})

In [53]:
type(table)

pandas.core.frame.DataFrame

In [54]:
table.reset_index()

,A,C,D,E
0,bar,large,5.500000,7.500000
1,bar,small,5.500000,8.500000
2,foo,large,2.000000,4.500000
3,foo,small,2.333333,4.333333


'''
    피벗 함수 이해를 위한 전처리
'''

In [56]:
pre_sale.dtypes

지역명         object
규모구분        object
연도           int64
월            int64
분양가격(㎡)     object
분양가격       float64
평당분양가격     float64
dtype: object

In [57]:
pre_sale.head()

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
0,서울,전체,2015,10,5841,5841.0,19275.3
1,서울,전용면적 60㎡이하,2015,10,5652,5652.0,18651.6
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882,5882.0,19410.6
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721,5721.0,18879.3
4,서울,전용면적 102㎡초과,2015,10,5879,5879.0,19400.7


In [58]:
#전용면적 60㎡이하 인 데이터만 검색

a = pre_sale.loc[pre_sale['규모구분'] == "전용면적 60㎡이하"]
a

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
1,서울,전용면적 60㎡이하,2015,10,5652,5652.0,18651.6
6,인천,전용면적 60㎡이하,2015,10,3488,3488.0,11510.4
11,경기,전용면적 60㎡이하,2015,10,3126,3126.0,10315.8
16,부산,전용면적 60㎡이하,2015,10,2950,2950.0,9735.0
21,대구,전용면적 60㎡이하,2015,10,2614,2614.0,8626.2
...,...,...,...,...,...,...,...
2866,전북,전용면적 60㎡이하,2018,7,2655,2655.0,8761.5
2871,전남,전용면적 60㎡이하,2018,7,2228,2228.0,7352.4
2876,경북,전용면적 60㎡이하,2018,7,2471,2471.0,8154.3
2881,경남,전용면적 60㎡이하,2018,7,2748,2748.0,9068.4


In [59]:
a.shape

(578, 7)

In [60]:
b = a.loc[a['연도'] == 2015]
b

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
1,서울,전용면적 60㎡이하,2015,10,5652,5652.0,18651.6
6,인천,전용면적 60㎡이하,2015,10,3488,3488.0,11510.4
11,경기,전용면적 60㎡이하,2015,10,3126,3126.0,10315.8
16,부산,전용면적 60㎡이하,2015,10,2950,2950.0,9735.0
21,대구,전용면적 60㎡이하,2015,10,2614,2614.0,8626.2
26,광주,전용면적 60㎡이하,2015,10,2253,2253.0,7434.9
31,대전,전용면적 60㎡이하,2015,10,2585,2585.0,8530.5
36,울산,전용면적 60㎡이하,2015,10,2422,2422.0,7992.6
41,세종,전용면적 60㎡이하,2015,10,2572,2572.0,8487.6
46,강원,전용면적 60㎡이하,2015,10,2286,2286.0,7543.8


In [61]:
b.shape

(51, 7)

In [62]:
b['평당분양가격'].mean()

8951.525000000001

"평당분양가격"을 "규모구분"와 "연도" 별로 평균값 구하기

In [63]:
pre_sale

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
0,서울,전체,2015,10,5841,5841.0,19275.3
1,서울,전용면적 60㎡이하,2015,10,5652,5652.0,18651.6
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882,5882.0,19410.6
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721,5721.0,18879.3
4,서울,전용면적 102㎡초과,2015,10,5879,5879.0,19400.7
...,...,...,...,...,...,...,...
2885,제주,전체,2018,7,3334,3334.0,11002.2
2886,제주,전용면적 60㎡이하,2018,7,NaN,NaN,NaN
2887,제주,전용면적 60㎡초과 85㎡이하,2018,7,3356,3356.0,11074.8
2888,제주,전용면적 85㎡초과 102㎡이하,2018,7,3226,3226.0,10645.8


In [64]:
#pre_sale.pivot_table(values=None, index=None, columns=None)

pre_sale.pivot_table(values="평당분양가격", index="규모구분", columns="연도")

연도,2015,2016,2017,2018
규모구분,,,,
전용면적 102㎡초과,9837.226667,10388.728272,11334.484615,11235.314563
전용면적 60㎡이하,8951.525000,9398.876119,10246.133333,10911.633333
전용면적 60㎡초과 85㎡이하,8891.817647,9295.986765,9825.000000,10463.662185
전용면적 85㎡초과 102㎡이하,9518.506250,10122.355435,10540.402454,11483.085542
전체,8893.047059,9293.042647,9901.270588,10563.022689


규모 구분이 전체로 되어 있는 금액으로 연도별 변동금액 알아보기

In [65]:
all = pre_sale.loc[pre_sale['규모구분'] == '전체']
all

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
0,서울,전체,2015,10,5841,5841.0,19275.3
5,인천,전체,2015,10,3163,3163.0,10437.9
10,경기,전체,2015,10,3138,3138.0,10355.4
15,부산,전체,2015,10,3112,3112.0,10269.6
20,대구,전체,2015,10,2682,2682.0,8850.6
...,...,...,...,...,...,...,...
2865,전북,전체,2018,7,2333,2333.0,7698.9
2870,전남,전체,2018,7,2357,2357.0,7778.1
2875,경북,전체,2018,7,2674,2674.0,8824.2
2880,경남,전체,2018,7,2696,2696.0,8896.8


In [66]:
# 평당분양가격, 지역명, 연도 관련해 pivot_table()

yearUp = all.pivot_table(values='평당분양가격', index='지역명', columns='연도')
yearUp

연도,2015,2016,2017,2018
지역명,,,,
강원,7159.9,7011.125,7126.8,7640.914286
경기,10377.4,11220.000,11850.0,12880.842857
경남,7585.6,7847.950,8119.8,8894.442857
경북,7170.9,7360.650,7794.9,8341.928571
광주,8052.0,8898.725,9463.5,9637.885714
대구,9232.3,10310.025,11455.8,11737.157143
대전,8098.2,8501.625,9044.7,9604.414286
부산,10307.0,10429.925,11577.9,12756.385714
서울,19725.2,20663.500,21375.9,22328.742857


In [67]:
# 지역명 index 를 하나의 컬럼으로 변경

yearUp.reset_index()

연도,지역명,2015,2016,2017,2018
0,강원,7159.9,7011.125,7126.8,7640.914286
1,경기,10377.4,11220.000,11850.0,12880.842857
2,경남,7585.6,7847.950,8119.8,8894.442857
3,경북,7170.9,7360.650,7794.9,8341.928571
4,광주,8052.0,8898.725,9463.5,9637.885714
5,대구,9232.3,10310.025,11455.8,11737.157143
6,대전,8098.2,8501.625,9044.7,9604.414286
7,부산,10307.0,10429.925,11577.9,12756.385714
8,서울,19725.2,20663.500,21375.9,22328.742857
9,세종,8750.5,8860.500,9135.3,10374.728571


In [68]:
yearUp.dtypes

연도
2015    float64
2016    float64
2017    float64
2018    float64
dtype: object

In [69]:
# 2018년도에서 2015년도 변동액 연산해 보기

yearUp['변동액'] = yearUp[2018]-yearUp[2015]
yearUp

연도,2015,2016,2017,2018,변동액
지역명,,,,,
강원,7159.9,7011.125,7126.8,7640.914286,481.014286
경기,10377.4,11220.000,11850.0,12880.842857,2503.442857
경남,7585.6,7847.950,8119.8,8894.442857,1308.842857
경북,7170.9,7360.650,7794.9,8341.928571,1171.028571
광주,8052.0,8898.725,9463.5,9637.885714,1585.885714
대구,9232.3,10310.025,11455.8,11737.157143,2504.857143
대전,8098.2,8501.625,9044.7,9604.414286,1506.214286
부산,10307.0,10429.925,11577.9,12756.385714,2449.385714
서울,19725.2,20663.500,21375.9,22328.742857,2603.542857


In [70]:
# 소수 데이터들을 정수화 시킬수 있는 설정 + 천단위 쉼표 처리 추가

pd.options.display.float_format = '{:,.0f}'.format

In [1]:
yearUp

NameError: name 'yearUp' is not defined

In [72]:
'''
? 지역별 평당 분양가격 합계 구하기
피벗의 인수값 : 평당분양가격, 규모구분, ?
? 
'''
pre_sale.pivot_table("평당분양가격", "규모구분", "지역명")

지역명,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
규모구분,,,,,,,,,,,,,,,,,
전용면적 102㎡초과,"7,899","13,647","9,226","8,616","10,296","12,334","14,282","12,590","22,058","9,292","9,974","13,836","7,568","7,842","10,416","8,478","7,933"
전용면적 60㎡이하,"7,195","12,385","8,343","7,661","8,518","11,285","8,798","10,707","22,046","9,107","8,965","10,870","6,991","7,232","14,700","7,782","6,969"
전용면적 60㎡초과 85㎡이하,"7,228","11,691","8,110","7,690","9,176","10,931","8,807","11,191","20,781","9,272","10,641","10,748","6,968","6,891","9,943","7,504","7,044"
전용면적 85㎡초과 102㎡이하,"7,686","12,130","9,822","8,776","9,296","9,589","9,037","11,039","23,754","9,245","8,861","11,267","7,882","7,770","10,739","9,090","8,171"
전체,"7,197","11,706","8,137","7,696","9,167","10,897","8,880","11,295","21,169","9,263","10,619","10,739","6,982","6,902","10,253","7,494","6,994"


### avg15.09.csv 파일 전처리

별도의 정제 없이 file을 read해서 보기 - 단순 확인 후에 개선 사항 찾기

In [73]:
df = pd.read_csv("aptPriceDataSet/avg15.09.csv", encoding="utf-8")
df

FileNotFoundError: [Errno 2] No such file or directory: 'aptPriceDataSet/avg15.09.csv'

별도의 정제 없이 file을 read해서 보기 - skiprows 속성을 이용하여 해당 row 삭제 하기

In [73]:
# avg15.09.csv로 DataFrame 객체 생성하기

df = pd.read_csv("aptPriceDataSet/avg15.09.csv", encoding="utf-8", skiprows=1)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,구분,NaN,2013년,2014년,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(단위: 천원/3.3㎡)
1,시도,시군구,12월,1월,2월,3월,4월,5월,6월,7월,...,3월,4월,5월,6월,7월,8월,9월,전월비,전년말비,전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
5,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
6,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
7,NaN,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
8,NaN,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
9,NaN,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   9 non-null      object
 1   Unnamed: 1   15 non-null     object
 2   Unnamed: 2   23 non-null     object
 3   Unnamed: 3   23 non-null     object
 4   Unnamed: 4   22 non-null     object
 5   Unnamed: 5   22 non-null     object
 6   Unnamed: 6   22 non-null     object
 7   Unnamed: 7   22 non-null     object
 8   Unnamed: 8   22 non-null     object
 9   Unnamed: 9   22 non-null     object
 10  Unnamed: 10  22 non-null     object
 11  Unnamed: 11  22 non-null     object
 12  Unnamed: 12  22 non-null     object
 13  Unnamed: 13  22 non-null     object
 14  Unnamed: 14  22 non-null     object
 15  Unnamed: 15  23 non-null     object
 16  Unnamed: 16  22 non-null     object
 17  Unnamed: 17  22 non-null     object
 18  Unnamed: 18  22 non-null     object
 19  Unnamed: 19  22 non-null     ob

In [75]:
df.dtypes

Unnamed: 0     object
Unnamed: 1     object
Unnamed: 2     object
Unnamed: 3     object
Unnamed: 4     object
Unnamed: 5     object
Unnamed: 6     object
Unnamed: 7     object
Unnamed: 8     object
Unnamed: 9     object
Unnamed: 10    object
Unnamed: 11    object
Unnamed: 12    object
Unnamed: 13    object
Unnamed: 14    object
Unnamed: 15    object
Unnamed: 16    object
Unnamed: 17    object
Unnamed: 18    object
Unnamed: 19    object
Unnamed: 20    object
Unnamed: 21    object
Unnamed: 22    object
Unnamed: 23    object
Unnamed: 24    object
Unnamed: 25    object
Unnamed: 26    object
dtype: object

In [76]:
df.shape

(23, 27)

In [77]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,구분,NaN,2013년,2014년,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(단위: 천원/3.3㎡)
1,시도,시군구,12월,1월,2월,3월,4월,5월,6월,7월,...,3월,4월,5월,6월,7월,8월,9월,전월비,전년말비,전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477


In [78]:
year = df.iloc[ [1, 2, 3], [1, 2, 3]] 
year

,Unnamed: 1,Unnamed: 2,Unnamed: 3
1,시군구,12월,1월
2,NaN,"8,059","8,130"
3,NaN,"18,189","17,925"


In [79]:
year.columns=[['구분', "월", "월"]]

In [80]:
year

,구분,월,월
1,시군구,12월,1월
2,NaN,"8,059","8,130"
3,NaN,"18,189","17,925"


In [81]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,구분,NaN,2013년,2014년,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(단위: 천원/3.3㎡)
1,시도,시군구,12월,1월,2월,3월,4월,5월,6월,7월,...,3월,4월,5월,6월,7월,8월,9월,전월비,전년말비,전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
5,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
6,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
7,NaN,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
8,NaN,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
9,NaN,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722


In [82]:
month = df.iloc[1]
month

Unnamed: 0        시도
Unnamed: 1       시군구
Unnamed: 2       12월
Unnamed: 3        1월
Unnamed: 4        2월
Unnamed: 5        3월
Unnamed: 6        4월
Unnamed: 7        5월
Unnamed: 8        6월
Unnamed: 9        7월
Unnamed: 10       8월
Unnamed: 11       9월
Unnamed: 12      10월
Unnamed: 13      11월
Unnamed: 14      12월
Unnamed: 15       1월
Unnamed: 16       2월
Unnamed: 17       3월
Unnamed: 18       4월
Unnamed: 19       5월
Unnamed: 20       6월
Unnamed: 21       7월
Unnamed: 22       8월
Unnamed: 23       9월
Unnamed: 24      전월비
Unnamed: 25     전년말비
Unnamed: 26    전년동월비
Name: 1, dtype: object

python의 enumerate() 함수의 메뉴얼 참조 필수

year와 month의 데이터를 결합해서 "2014년 4월" 형태로 가공 필수

1. join()/year[index]/month[index]...
2. 2014 이후 부터 2015년 이전까지 2014로 채우기
3. 2015년 이후는 2015로 채우기

In [83]:
# 문자열 가공

v = 'e'.join(["가", "나"])
v

'가e나'

2014년 데이터 : 2014년 1월~12월 : 2014년 몇월 <br>
2015년 데이터 : 2015년 1월~12월 : 2015년 몇월 <br>    

In [84]:
year = df.iloc[0]
year

Unnamed: 0                구분
Unnamed: 1               NaN
Unnamed: 2             2013년
Unnamed: 3             2014년
Unnamed: 4               NaN
Unnamed: 5               NaN
Unnamed: 6               NaN
Unnamed: 7               NaN
Unnamed: 8               NaN
Unnamed: 9               NaN
Unnamed: 10              NaN
Unnamed: 11              NaN
Unnamed: 12              NaN
Unnamed: 13              NaN
Unnamed: 14              NaN
Unnamed: 15            2015년
Unnamed: 16              NaN
Unnamed: 17              NaN
Unnamed: 18              NaN
Unnamed: 19              NaN
Unnamed: 20              NaN
Unnamed: 21              NaN
Unnamed: 22              NaN
Unnamed: 23              NaN
Unnamed: 24              NaN
Unnamed: 25              NaN
Unnamed: 26    (단위: 천원/3.3㎡)
Name: 0, dtype: object

In [85]:
for i, y in enumerate(year):
    
    if i > 2 and i < 15:
        year[i] = ' '.join(['2014년', month[i]])
    elif i >= 15:
        year[i] = ' '.join(['2015년', month[i]])
    elif i == 2:
        year[i] = ' '.join([year[i], month[i]])
    elif i == 1:
        year[i] = '시구군'

year

Unnamed: 0              구분
Unnamed: 1             시구군
Unnamed: 2       2013년 12월
Unnamed: 3        2014년 1월
Unnamed: 4        2014년 2월
Unnamed: 5        2014년 3월
Unnamed: 6        2014년 4월
Unnamed: 7        2014년 5월
Unnamed: 8        2014년 6월
Unnamed: 9        2014년 7월
Unnamed: 10       2014년 8월
Unnamed: 11       2014년 9월
Unnamed: 12      2014년 10월
Unnamed: 13      2014년 11월
Unnamed: 14      2014년 12월
Unnamed: 15       2015년 1월
Unnamed: 16       2015년 2월
Unnamed: 17       2015년 3월
Unnamed: 18       2015년 4월
Unnamed: 19       2015년 5월
Unnamed: 20       2015년 6월
Unnamed: 21       2015년 7월
Unnamed: 22       2015년 8월
Unnamed: 23       2015년 9월
Unnamed: 24      2015년 전월비
Unnamed: 25     2015년 전년말비
Unnamed: 26    2015년 전년동월비
Name: 0, dtype: object

df 확인 후에 column에 적용하기

In [86]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
1,시도,시군구,12월,1월,2월,3월,4월,5월,6월,7월,...,3월,4월,5월,6월,7월,8월,9월,전월비,전년말비,전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
5,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
6,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
7,NaN,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
8,NaN,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
9,NaN,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722


In [87]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26'],
      dtype='object')

In [88]:
df.columns = year
df

,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
0,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
1,시도,시군구,12월,1월,2월,3월,4월,5월,6월,7월,...,3월,4월,5월,6월,7월,8월,9월,전월비,전년말비,전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
5,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
6,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
7,NaN,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
8,NaN,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
9,NaN,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722


In [89]:
df.head(3)

,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
0,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
1,시도,시군구,12월,1월,2월,3월,4월,5월,6월,7월,...,3월,4월,5월,6월,7월,8월,9월,전월비,전년말비,전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207


In [90]:
df.drop(df.index[[0,1]])

,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
5,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
6,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
7,NaN,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
8,NaN,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
9,NaN,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722
10,NaN,NaN,"8,151","8,355","8,597","8,552","8,585","8,606","8,669","8,648",...,"8,692","8,858","8,967","8,942","8,984","9,058","9,023",-36,325,352
11,경기,NaN,"10,855","10,855","10,791","10,784","10,876","10,646","10,266","10,124",...,"10,469","10,684","10,685","10,573","10,518","10,573","10,341",-232,-38,-160


In [91]:
df.head()

,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
0,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
1,시도,시군구,12월,1월,2월,3월,4월,5월,6월,7월,...,3월,4월,5월,6월,7월,8월,9월,전월비,전년말비,전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477


In [92]:
df.drop(df.index[[0,1]], inplace=True)

In [93]:
df

,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
5,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
6,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
7,NaN,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
8,NaN,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
9,NaN,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722
10,NaN,NaN,"8,151","8,355","8,597","8,552","8,585","8,606","8,669","8,648",...,"8,692","8,858","8,967","8,942","8,984","9,058","9,023",-36,325,352
11,경기,NaN,"10,855","10,855","10,791","10,784","10,876","10,646","10,266","10,124",...,"10,469","10,684","10,685","10,573","10,518","10,573","10,341",-232,-38,-160


index 재정렬

In [94]:
df = df.reset_index()
df.head()

,index,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
0,2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
1,3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
2,4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
3,5,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
4,6,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131


In [95]:
del df['index']
df

,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
0,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
1,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
2,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
3,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
4,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
5,NaN,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
6,NaN,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
7,NaN,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722
8,NaN,NaN,"8,151","8,355","8,597","8,552","8,585","8,606","8,669","8,648",...,"8,692","8,858","8,967","8,942","8,984","9,058","9,023",-36,325,352
9,경기,NaN,"10,855","10,855","10,791","10,784","10,876","10,646","10,266","10,124",...,"10,469","10,684","10,685","10,573","10,518","10,573","10,341",-232,-38,-160


In [96]:
df['구분'].fillna("", inplace=True)
df

,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
0,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
1,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
2,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
3,,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
4,,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
5,,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
6,,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
7,,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722
8,,NaN,"8,151","8,355","8,597","8,552","8,585","8,606","8,669","8,648",...,"8,692","8,858","8,967","8,942","8,984","9,058","9,023",-36,325,352
9,경기,NaN,"10,855","10,855","10,791","10,784","10,876","10,646","10,266","10,124",...,"10,469","10,684","10,685","10,573","10,518","10,573","10,341",-232,-38,-160


In [97]:
df['시구군'].fillna("",inplace=True)
df

,구분,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
0,전국,,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
1,서울,,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
2,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
3,,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
4,,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
5,,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
6,,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
7,,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722
8,,,"8,151","8,355","8,597","8,552","8,585","8,606","8,669","8,648",...,"8,692","8,858","8,967","8,942","8,984","9,058","9,023",-36,325,352
9,경기,,"10,855","10,855","10,791","10,784","10,876","10,646","10,266","10,124",...,"10,469","10,684","10,685","10,573","10,518","10,573","10,341",-232,-38,-160


In [98]:
cat = df['구분'] 
city = df['시구군']

for i, y in enumerate(cat):
    if y == '6대광역시':
        df['시구군'][i] = city[i]
    elif y == '지방':
        df['시구군'][i] = city[i]
    else :
        df['시구군'][i]= ' '.join([y, city[i]])
        

del df['구분']

In [99]:
df

,시구군,2013년 12월,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,2014년 8월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
0,전국,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333","8,395",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
1,서울,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742","19,274",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
2,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457","9,411",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
3,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360","8,370",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
4,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974","9,973",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
5,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612","7,622",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
6,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
7,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153","8,493",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722
8,,"8,151","8,355","8,597","8,552","8,585","8,606","8,669","8,648","8,700",...,"8,692","8,858","8,967","8,942","8,984","9,058","9,023",-36,325,352
9,경기,"10,855","10,855","10,791","10,784","10,876","10,646","10,266","10,124","10,134",...,"10,469","10,684","10,685","10,573","10,518","10,573","10,341",-232,-38,-160
